In [2]:
import torch

print(torch.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # esto por si hay disponible una GPU si no usa la CPU
print("CUDA:", torch.cuda.is_available())

2.9.0+cu126
CUDA: True


In [3]:
import zipfile
import os

zip_file_path = '/content/contaminante.zip'
extract_dir = '/content/'

os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"'{zip_file_path}' decompressed to '{extract_dir}'")

'/content/contaminante.zip' decompressed to '/content/'


## Integridad de los datos

In [4]:
import os

base_path = r"/content/contaminante"
data_types = ["train", "valid", "test"]

for split in data_types:
    images_path = os.path.join(base_path, split, "images")
    labels_path = os.path.join(base_path, split, "labels")

    num_images = len([
        f for f in os.listdir(images_path)
        if f.lower().endswith((".png", ".jpg", ".jpeg"))
    ])

    num_labels = len(os.listdir(labels_path))

    print(f"{split.upper()}:")
    print(f"  Imágenes: {num_images}")
    print(f"  Labels:   {num_labels}")


TRAIN:
  Imágenes: 3704
  Labels:   3704
VALID:
  Imágenes: 1236
  Labels:   1236
TEST:
  Imágenes: 1232
  Labels:   1232


- dataset personalizado para el uso de multi etiqueta en una sola imagen

In [ ]:
import os
import torch
from torch.utils.data import Dataset
from PIL import Image


class ContaminantDataset(Dataset):
    def __init__(self, images_dir, labels_dir, transform=None):
        self.images_dir = images_dir
        self.labels_dir = labels_dir
        self.transform = transform
        self.image_files = sorted(os.listdir(images_dir))
        self.class_to_idx = {
            "plastico": 0,
            "vidrio": 1,
            "metal": 2,
            "organico": 3
        }

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]

        # Imagen
        img_path = os.path.join(self.images_dir, img_name)
        image = Image.open(img_path).convert("RGB")

        # Label multi-label
        label_path = os.path.join(
            self.labels_dir,
            img_name.replace(".jpg", ".txt")
        )

        label = torch.zeros(len(self.class_to_idx), dtype=torch.float32)

        with open(label_path, "r") as f:
            for line in f:
                parts = line.strip().split()
                class_id = int(parts[0])
                label[class_id] = 1

        if self.transform:
            image = self.transform(image)

        return image, label


- preprosesado de loas imagenes para ser usadas por los modelos

In [ ]:
from torchvision import transforms

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

- construccion de las retas a los apartados del dataset

In [ ]:
train_dataset = ContaminantDataset(
    base_path +"/train/images",
    base_path +"/train/labels",
    transform=train_transform
)

val_dataset = ContaminantDataset(
    base_path +"/valid/images",
    base_path +"/valid/labels",
    transform=val_transform
)

test_dataset = ContaminantDataset(
    base_path +"/test/images",
    base_path +"/test/labels",
    transform=test_transform
)


- verificasion de la catidad de los datos con respecto a catidad de archivos

In [ ]:
print("este es el tamaño del apartado de entrenamiento: " ,len(train_dataset))
print("este es el tamaño del apartado de validacion: ",len(val_dataset))
print("este es el tamaño del apartado de prueba: ", len(test_dataset))


este es el tamaño del apartado de entrenamiento:  3704
este es el tamaño del apartado de validacion:  1236
este es el tamaño del apartado de prueba:  1232


In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False
)

test_loader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False
)


## Cargar ResNet18 preentrenada

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

NUM_CLASSES = 4  # ajusta a tus contaminantes reales

# OPCIÓN 1: Usando DEFAULT (recomendado para compatibilidad futura)
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

# OPCIÓN 2: Específico de ImageNet (equivalente a pretrained=True)
# model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

# OPCIÓN 3: Sin pesos preentrenados
# model = models.resnet18(weights=None)

# Cambiamos la última capa
model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

- Loss y Optimizer

In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

- Verificación rápida del DataLoader

In [ ]:
images, labels = next(iter(train_loader))

print(images.shape)   # (batch, 3, 224, 224)
print(labels.shape)   # (batch, NUM_CLASSES)


torch.Size([32, 3, 224, 224])
torch.Size([32, 4])


- Training loop

In [ ]:
for epoch in range(2):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1} - Train Loss: {running_loss/len(train_loader):.4f}")


Epoch 1 - Train Loss: 0.1341
Epoch 2 - Train Loss: 0.0537


In [ ]:
import numpy as np
import torch
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    hamming_loss
)


model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)

        outputs = model(images)
        probs = torch.sigmoid(outputs)
        preds = (probs > 0.5).cpu().numpy()

        all_preds.append(preds)
        all_labels.append(labels.numpy())

all_preds = np.vstack(all_preds)
all_labels = np.vstack(all_labels)



exact_match = accuracy_score(all_labels, all_preds)



hamming_acc = 1 - hamming_loss(all_labels, all_preds)



f1_micro = f1_score(all_labels, all_preds, average="micro")
f1_macro = f1_score(all_labels, all_preds, average="macro")


precision = precision_score(all_labels, all_preds, average="macro")
recall = recall_score(all_labels, all_preds, average="macro")




print("Exact Match Accuracy :", exact_match)
print("Hamming Accuracy     :", hamming_acc)
print("F1-score (micro)     :", f1_micro)
print("F1-score (macro)     :", f1_macro)
print("Precision (macro)    :", precision)
print("Recall (macro)       :", recall)



Exact Match Accuracy : 0.790453074433657
Hamming Accuracy     : 0.9476132686084142
F1-score (micro)     : 0.8276779773785762
F1-score (macro)     : 0.20691949434464404
Precision (macro)    : 0.19559748427672957
Recall (macro)       : 0.2196327683615819


c:\Proyectos\ServicioSocial\venv\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Proyectos\ServicioSocial\venv\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Proyectos\ServicioSocial\venv\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [ ]:
print(all_labels.sum(axis=0))


[708.   0.   0.   0.]
